### Importing Modules

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')

2024-02-23 13:37:14.096905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 13:37:14.097045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 13:37:14.269026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Reading Dataset

In [2]:
dataset,info=tfds.load(name='mnist',as_supervised=True,with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


# Data Preprocessing

In [3]:
train,test=dataset['train'],dataset['test']

### Scaling Data

In [4]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image=image/255.
  return image,label

train_scaled=train.map(scale)
test_scaled=test.map(scale)

### Shuffling Data

In [5]:
BUFFER_SIZE=10000

In [6]:
shuffled=train_scaled.shuffle(BUFFER_SIZE)

### Separating Validation and Train Data

In [7]:
validation_count=tf.cast(0.1*info.splits['train'].num_examples,tf.int64)
test_size=tf.cast(info.splits['test'].num_examples,tf.int64)

In [8]:
validation=shuffled.take(validation_count)
train=shuffled.skip(validation_count)

### Creating Batches For Increasing Training Time

In [9]:
BATCH_SIZE=100

In [10]:
train=train.batch(BATCH_SIZE)

In [11]:
validation=validation.batch(validation_count)

In [12]:
validation_inputs,validation_target=next(iter(validation))

In [13]:
test=test.batch(test_size)

# Model Creation

In [14]:
input_size=784
output_size=10
hidden_layer_units=100
model=tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28,28,1)),
        tf.keras.layers.Dense(hidden_layer_units,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_units,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_units,activation='relu'),
        tf.keras.layers.Dense(output_size,activation='softmax'),
    ]
)

In [15]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

**3 type of loss function for categorical types : 1) binary crossentropy 2)categorical crossentropy 3)sparse categorical crossentropy**


# Model Fitting

In [16]:
EPOCHS=5

In [17]:
model.fit(train,epochs=EPOCHS,validation_data=(validation_inputs,validation_target),verbose=2)

Epoch 1/5
540/540 - 7s - loss: 0.3204 - accuracy: 0.9066 - val_loss: 0.1536 - val_accuracy: 0.9538 - 7s/epoch - 13ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1307 - accuracy: 0.9602 - val_loss: 0.1349 - val_accuracy: 0.9578 - 4s/epoch - 8ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0952 - accuracy: 0.9710 - val_loss: 0.0809 - val_accuracy: 0.9757 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0729 - accuracy: 0.9782 - val_loss: 0.0704 - val_accuracy: 0.9792 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0591 - accuracy: 0.9821 - val_loss: 0.0623 - val_accuracy: 0.9792 - 4s/epoch - 8ms/step


# Evaluating model With Test Data

In [18]:
model_loss,model_accuracy=model.evaluate(test)

1/1 [==============================] - 1s 854ms/step - loss: 17.2300 - accuracy: 0.9703


In [19]:
print(f"model loss {model_loss:.2f}: model accuracy {model_accuracy*100:.2f}%")

model loss 17.23: model accuracy 97.03%
